# LSTM诗词生成实验

文本生成是自然语言处理领域的一个重要研究方向，研究人员期待有一天计算机能够像人类一样会表达，能够撰写出高质量的自然语言文本。


## 实验目的

本章的主要内容就是基于 MindSpore 实现文本生成，通过本实验，学员可以学习LSTM网络的使用，文本的预处理操作。熟悉循环神经网络的结构，文本生成的过程以及网络模型的训练和预测，掌握 MindSpore 的相关操作等。 


## 实验环境

ModelArts Notebook环境，目前仅支持GPU/Ascend，具体参考实验正文。


## 数据处理

本文使用了2930首五言律诗作为训练数据集，每一行数据为一首诗，这里使用的诗的作者和诗名都已经被删去。

数据集保存在文件 `poetry.txt` 中，其中的部分古诗如下：

```text
山山浮翠远，处处落红深。独立柴门外，长歌托素心。
翠华重幸日，瑞气霭龙牙。元老频承宠，宸章特赐嘉。
调和归静穆，暑雨绝咨嗟。共仰明良会，唐虞岂有加。
一字同华衮，天香遍齿牙。光辉悬日月，矍铄入褒嘉。
...
```


### 数据集下载

使用 `download` 接口下载数据集，并将下载后的数据集自动解压到当前目录下。数据下载之前需要使用 `pip install download` 安装 `download` 包。


In [1]:
from download import download

url = "https://ascend-professional-construction-dataset.obs.cn-north-4.myhuaweicloud.com:443/NLP/LSTM_generate_poem_data.zip"

download(url, "./", kind="zip", replace=True)


file_sizes: 100%|███████████████████████████| 3.20M/3.20M [00:03<00:00, 829kB/s]
Extracting zip file...
Successfully downloaded / unzipped to ./


'./'

### 预处理

先按行读取数据集，再使用 `rstrip` 方法去除后缀字符，然后过滤掉含有特殊字符的文本，然后用 `word2vec` 将文本词向量化。

### Work2Vec

Word2Vec 是一种将词转为向量的方法，是在2013年的论文 [Efficient Estimation of Word Representations inVector Space](https://arxiv.org/pdf/1301.3781.pdf) 中提出的，作者来自google。

比较早出现的将词转为向量的方法是独热编码（One-Hot）。使用独热编码构建独热向量很容易，但通常不是一个好的选择。一个主要原因是独热向量不能准确表达不同词之间的相似度，而且在大量数据的情况下会出现数据的维度灾难。

Word2Vec 方法刚好克服了独热编码的缺陷。它将每个词映射到一个固定长度（长度为可设置的超参数）的向量，这些向量能很好地表达不同词之间的相似性和类比关系。该方法包含两种模型算法：skip-gram 和 CBOW，它们的最大区别是 skip-gram 是通过中心词去预测中心词周围的词，而 CBOW 是通过周围的词去预测中心词。

此方法需要通过 `gensim.models.word2vec` 来使用，上述的两种训练模型可以通过超参数 `sg` 来设置，默认为 CBOW。这里介绍 Work2Vec 的原因是它可以替换下文的模型的 Embedding 层。

> `gensim` 库需手动安装，命令如下：`pip install gensim`

具体情况见如下代码：


In [2]:
import os
import pickle

import mindspore as ms
from gensim.models.word2vec import Word2Vec

dataset_path = './LSTM_generate_poem_data/poetry_5.txt'
vec_params_file = 'vec_params_5.pkl'  # 保存词向量训练后的权重文件名字

ms.set_context(device_target='CPU', mode=ms.PYNATIVE_MODE, device_id=0)

def data_pre(dataset_path, vec_params_file):
    # 特殊字符列表
    forbidden_words = ['（', '）', '(', ')', '__', '《', '》', '【', '】', '[', ']']
    poetry_src = []
    with open(dataset_path, encoding='utf-8') as f:
        lines = f.readlines()
    for poem in lines:
        forbidden_poem = [word in poem for word in forbidden_words]
        # 过滤掉含有特殊字符的诗句
        if sum(forbidden_poem) > 0:
            continue
        # 去除行尾换行符
        poem = poem.rstrip()
        poetry_src.append(poem)

    if os.path.exists(vec_params_file):
        return poetry_src, pickle.load(open(vec_params_file, "rb"))

    model = Word2Vec(poetry_src, vector_size=100, min_count=1, workers=6)
    pickle.dump((model.syn1neg, model.wv.key_to_index, model.wv.index_to_key), open(vec_params_file, "wb"))

    return poetry_src, (model.syn1neg, model.wv.key_to_index, model.wv.index_to_key)

all_data, (word_vec, word_2_index, index_2_word) = data_pre(dataset_path, vec_params_file)
vocab_size, embedding_dim = word_vec.shape
print('vocab_size: ', vocab_size)

vocab_size:  4436


### 加载数据集

完成预处理操作后，需将其加入到数据集处理流水线中。类方法 `PoetryDataGenerator` 将数据拆分为模型的输入数据和用于计算loss的标签，并转换为 Tensor 格式，`__getitem__` 方法将数据变为可迭代对象。函数 `create_poetry_dataset` 是将可迭代的数据使用 `GeneratorDataset` 封装成MindSpore 的 dataset 对象，然后可以使用dataset 对象的 `get_batch_size` 属性获得数据集的大小 ，使用 `batch` 批操作将数据批量化。


In [3]:
import mindspore.dataset as ds

class PoetryDataGenerator(object):
    def __init__(self, all_data, word_vec, word_2_index):
        self.all_data = all_data
        self.word_vec = word_vec
        self.word_2_index = word_2_index

    # 获取一条数据, 并作处理
    def __getitem__(self, index):
        a_poetry_words = self.all_data[index]
        a_poetry_index = [self.word_2_index[word] for word in a_poetry_words]
        xs_index = a_poetry_index[:-1]
        ys_index = a_poetry_index[1:]
        xs_embedding = self.word_vec[xs_index]

        return Tensor(xs_embedding), Tensor(ys_index, ms.int32)

    def __len__(self):
        return len(self.all_data)

def create_poetry_dataset(batch_size, all_data, word_vec, word_2_index):
    dt = PoetryDataGenerator(all_data, word_vec, word_2_index)
    de = ds.GeneratorDataset(dt, ["inputs", "label"], shuffle=True)
    size = de.get_dataset_size()
    de = de.batch(batch_size, drop_remainder=True)
    return de, size

In [4]:
from mindspore import Tensor

batch_size = 64

dataset, size = create_poetry_dataset(batch_size, all_data, word_vec, word_2_index)
total_step = dataset.get_dataset_size()
data1 = next(dataset.create_dict_iterator())
data1_shape = data1["inputs"].shape

print('Input data size:', data1_shape)
print('Size:', size)
print('Batchsize:', batch_size)
print('Total_step:', total_step)

Input data size: (64, 23, 100)
Size: 2930
Batchsize: 64
Total_step: 45


## 模型构建

完成数据集的处理后，我们设计用于文本生成的模型结构。一般情况下首先需要使用`nn.Embedding`层加载词向量；然后使用RNN循环神经网络做特征提取；最后将RNN连接至一个全连接层，即 `nn.Dense` ，将特征转化为需要预测的词表大小的结果，用于后续进行模型优化训练。整体模型结构如下：

```text
nn.Embedding -> nn.RNN -> nn.Dense
```

这里我们使用能够一定程度规避RNN梯度消失问题的变种 LSTM(Long short term memory) 做特征提取层。下面对模型进行详解：

### Embedding

Embedding 层又可称为 EmbeddingLookup 层，其作用是使用 index id 对权重矩阵对应id的向量进行查找，当输入为一个由 index id 组成的序列时，则查找并返回一个相同长度的矩阵，例如：

```text
embedding = nn.Embedding(1000, 100) # 词表大小(index的取值范围)为1000，表示向量的size为100
input shape: (1, 16)                # 序列长度为16
output shape：(1, 16, 100)
```

这里我们使用 Work2Vec 方法处理的数据来替代 Embedding 层，可以达到相同的效果。

### RNN(循环神经网络)

循环神经网络（Recurrent Neural Network, RNN）是一类以序列（sequence）数据为输入，在序列的演进方向进行递归（recursion）且所有节点（循环单元）按链式连接的神经网络。下图为RNN的一般结构：

![RNN-0](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/website-images/master/tutorials/application/source_zh_cn/nlp/images/0-RNN-0.png)

> 图示左侧为一个RNN Cell循环，右侧为RNN的链式连接平铺。实际上不管是单个RNN Cell还是一个RNN网络，都只有一个Cell的参数，在不断进行循环计算中更新。

由于RNN的循环特性，和自然语言文本的序列特性(句子是由单词组成的序列)十分匹配，因此被大量应用于自然语言处理研究中。下图为RNN的结构拆解：

![RNN](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/website-images/master/tutorials/application/source_zh_cn/nlp/images/0-RNN.png)

RNN单个Cell的结构简单，因此也造成了梯度消失(Gradient Vanishing)问题，具体表现为RNN网络在序列较长时，在序列尾部已经基本丢失了序列首部的信息。为了克服这一问题，LSTM(Long short term memory)被提出，通过门控机制(Gating Mechanism)来控制信息流在每个循环步中的留存和丢弃。下图为LSTM的结构拆解：

![LSTM](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/website-images/master/tutorials/application/source_zh_cn/nlp/images/0-LSTM.png)

本节我们选择LSTM变种而不是经典的RNN做特征提取，来规避梯度消失问题，并获得更好的模型效果。下面来看MindSpore中`nn.LSTM`对应的公式：

$$h_{0:t}, (h_t, c_t) = \text{LSTM}(x_{0:t}, (h_0, c_0))$$

这里`nn.LSTM`隐藏了整个循环神经网络在序列时间步(Time step)上的循环，送入输入序列、初始状态，即可获得每个时间步的隐状态(hidden state)拼接而成的矩阵，以及最后一个时间步对应的隐状态。我们使用最后的一个时间步的隐状态作为输入句子的编码特征，送入下一层。

> Time step：在循环神经网络计算的每一次循环，成为一个Time step。在送入文本序列时，一个Time step对应一个单词。因此在本例中，LSTM的输出$h_{0:t}$对应每个单词的隐状态集合，$h_t$和$c_t$对应最后一个单词对应的隐状态。

> 以上内容借鉴MindSpore官方教程的应用实践案例：[RNN实现情感分类](https://www.mindspore.cn/tutorials/application/zh-CN/master/nlp/sentiment_analysis.html#rnn%E5%AE%9E%E7%8E%B0%E6%83%85%E6%84%9F%E5%88%86%E7%B1%BB)

### Dense

在经过LSTM编码获取句子特征后，将其送入一个全连接层，即`nn.Dense`，将特征维度变换为需要预测的词表大小，然后使用 `ops.argmax` 选取最大的结果的位置，再对应到词表找到相应的词汇即为模型预测结果。

In [5]:
from mindspore import nn, ops, Tensor, save_checkpoint

hidden_dim = 128
n_layers = 2

class LSTMPoetryModel(nn.Cell):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, n_layers, batch_size):
        super().__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.batch_size = batch_size
        self.rnn = nn.LSTM(self.embedding_dim,
                           self.hidden_dim,
                           num_layers=self.n_layers,
                           batch_first=True)
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Dense(self.hidden_dim, self.vocab_size)

    def construct(self, inputs, h_0=None, c_0=None):
        if h_0 == None or c_0 == None:
            h_0 = Tensor(np.zeros((self.n_layers, self.batch_size, self.hidden_dim)), ms.float32)
            c_0 = Tensor(np.zeros((self.n_layers, self.batch_size, self.hidden_dim)), ms.float32)

        outputs, (h_0, c_0) = self.rnn(inputs, (h_0, c_0))
        outputs = self.dropout(outputs)
        outputs = ops.reshape(outputs, (outputs.shape[0] * outputs.shape[1], outputs.shape[2]))
        output = self.fc(outputs)
        return output, h_0, c_0

# 模型实例化
rnn_net = LSTMPoetryModel(vocab_size, embedding_dim, hidden_dim, n_layers, batch_size)


## 损失函数和优化器

完成模型主体构建后，然后选择损失函数和优化器。针对本案例使用的文本数据的特性，即一个多分类问题，这里选择交叉熵损失函数 `nn.CrossEntropyLoss()`，优化器选择 `nn.Adam`。


In [6]:
lr = 0.003

loss_fn = nn.CrossEntropyLoss()
opt = nn.Adam(rnn_net.trainable_params(), learning_rate=lr)


## 训练过程

训练过程主要是先前向计算损失，然后根据损失值计算梯度，再根据梯度更新模型参数，最后打印损失保存权重文件。


In [7]:
import time
import numpy as np

from mindspore import save_checkpoint
from mindspore import context

total_epoch = 1000

def forward_fn(datas):
    output, h_0, c_0 = rnn_net(datas["inputs"])
    loss = loss_fn(output, datas["label"].reshape(-1))
    return loss

grad_fn = ms.value_and_grad(forward_fn, None, rnn_net.trainable_params())

def train_step(data1):
    loss, grads = grad_fn(data1)
    opt(grads)
    return loss

def train():
    print('Start training!')
    epochs_loss = []
    for epoch in range(total_epoch):
        start = time.time()
        steps_loss = []
        for step, data in enumerate(dataset.create_dict_iterator()):
            start1 = time.time()
            loss = train_step(data)

            steps_loss.append(loss.asnumpy())
            end1 = time.time()
            if step % 10 == 0:
                print(f"Epoch:[{int(epoch+1):>3d}/{int(total_epoch):>3d}], "
                      f"step:[{int(step):>4d}/{int(total_step):>4d}], "
                      f"loss:{loss.asnumpy():>4f} , "
                      f"time:{(end1 - start1):>3f}s, "
                      f"lr:{lr:>6f}")
        epochs_loss.append(np.mean(steps_loss))
        end = time.time()
        print(f"Epoch:[{int(epoch + 1):>3d}/{int(total_epoch):>3d}], "
              f"loss:{epochs_loss[epoch]:>4f} , "
              f"time:{(end - start):>3f}s, ")
        if (epoch+1) % 10 == 0:
            save_checkpoint(rnn_net, f"./LSTM_poetry_{epoch+1}.ckpt")

    print('Training completed!')

train()


Start training!
Epoch:[  1/1000], step:[   0/  45], loss:8.397839 , time:2.591877s, lr:0.003000
Epoch:[  1/1000], step:[  10/  45], loss:7.154466 , time:0.057803s, lr:0.003000
Epoch:[  1/1000], step:[  20/  45], loss:6.991076 , time:0.057386s, lr:0.003000
Epoch:[  1/1000], step:[  30/  45], loss:6.842004 , time:0.072192s, lr:0.003000
Epoch:[  1/1000], step:[  40/  45], loss:6.909730 , time:0.058578s, lr:0.003000
Epoch:[  1/1000], loss:7.165276 , time:5.382530s, 
Epoch:[  2/1000], step:[   0/  45], loss:6.711904 , time:0.122232s, lr:0.003000
...
Epoch:[999/1000], step:[  40/  45], loss:1.877362 , time:0.069371s, lr:0.003000
Epoch:[999/1000], loss:1.907126 , time:3.458176s, 
Epoch:[1000/1000], step:[   0/  45], loss:1.786555 , time:0.115004s, lr:0.003000
Epoch:[1000/1000], step:[  10/  45], loss:1.939893 , time:0.073283s, lr:0.003000
Epoch:[1000/1000], step:[  20/  45], loss:2.090731 , time:0.070930s, lr:0.003000
Epoch:[1000/1000], step:[  30/  45], loss:1.893512 , time:0.070538s, lr:0.0

## 模型预测

最后我们设计一个预测函数，实现随机初始化一首诗的第一个字然后生成一首五言诗。具体代码如下:


In [8]:
def eval():
    result = ""
    word_index = np.random.randint(0, vocab_size, 1)[0]
    result += index_2_word[word_index]
    print('Start word:', result)

    h_0 = Tensor(np.zeros((n_layers, 1, hidden_dim)), ms.float32)
    c_0 = Tensor(np.zeros((n_layers, 1, hidden_dim)), ms.float32)

    for i in range(data1_shape[1]):
        inputs = Tensor([[word_vec[word_index]]])
        output, h_0, c_0 = rnn_net(inputs, h_0, c_0)
        word_index = int(ops.argmax(output))
        result += index_2_word[word_index]

    print('Predict result:', result)


In [9]:
from mindspore import load_checkpoint, load_param_into_net

param_dict = load_checkpoint('LSTM_poetry_1000.ckpt')
load_param_into_net(rnn_net, param_dict)
eval()

Start word: 斧
Predict result: 斧有三秋啼，团乔数其手。虽锋荡回问，或此者中边。
